In [11]:
from flask import Flask, render_template, jsonify
import requests
import time
from azure.storage.blob import BlobServiceClient as syncBlobServiceClient
import os
import asyncio

from azure.identity.aio import DefaultAzureCredential
from azure.storage.blob.aio import BlobServiceClient, BlobClient


In [12]:

# Azure Blob Storage connection string and container details
AZURE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=oishik;AccountKey=aleUFtKbyShYewMZZZ1vKx+iQK9tmfaqy3pkGdsB0Sv5a+MeiePSpii8s6JpOyWi1Yh7uXiNxW+s+AStq044EQ==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "tvi"

# Blob service client
blob_service_client = syncBlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)


In [14]:

# The Raspberry Pi IP address or hostname
raspberry_pi_url = "http://<raspberry-pi-ip>:5000/take_picture"


blob_list = container_client.list_blobs(name_starts_with='raspberry_pi_uploads')
image_blob_names = []
for blob in blob_list:
    image_blob_names.append(blob.name)
    # image_urls.append(f"https://oishik.blob.core.windows.net/{CONTAINER_NAME}/{blob.name}")

TypeError: object of type 'AsyncItemPaged' has no len()

In [57]:
from PIL import Image
from io import BytesIO
url = image_urls[0]
response = requests.get(url)
img = Image.open(BytesIO(response.content))

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000020CCB565EE0>

In [ ]:


class BlobSamples:
    def __init__(self):
        self.container_name = "sample-container"  # Replace with your actual container name

    async def download_blob_to_variable(self, blob_service_client: BlobServiceClient, container_name: str, blob_name: str):
        # Create the BlobClient to interact with the specific blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        try:
            # Download the blob asynchronously
            download_stream = await blob_client.download_blob()
            print(f"Blob {blob_name} downloaded to memory.")
            return download_stream
        except Exception as e:
            print(f"An error occurred while downloading the blob {blob_name}: {e}")

    async def download_multiple_blobs(self, blob_service_client: BlobServiceClient, container_name: str, blob_names: list):
        # Create a list of tasks for downloading multiple blobs concurrently
        download_tasks = [
            self.download_blob_to_variable(blob_service_client, container_name, blob_name)
            for blob_name in blob_names
        ]
        # Use asyncio.gather to run all download tasks concurrently
        downloaded_data = await asyncio.gather(*download_tasks)
        return downloaded_data

async def get_blobs(blob_names, azure_connection_string):
    sample = BlobSamples()

    # Create the BlobServiceClient asynchronously, using the connection string
    async with BlobServiceClient.from_connection_string(azure_connection_string) as blob_service_client:
        blob_data = await sample.download_multiple_blobs(
            blob_service_client=blob_service_client,
            container_name=sample.container_name, 
            blob_name=blob_names)
    return blob_data


In [19]:
await get_blobs(blob_names=image_blob_names, azure_connection_string=AZURE_CONNECTION_STRING)

An error occurred while downloading the blob raspberry_pi_uploads/1736779848.6032581(1).jpg: The specified container does not exist.
RequestId:a7ae467d-d01e-0017-5de1-664c71000000
Time:2025-01-15T00:10:02.7693407Z
ErrorCode:ContainerNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerNotFound</Code><Message>The specified container does not exist.
RequestId:a7ae467d-d01e-0017-5de1-664c71000000
Time:2025-01-15T00:10:02.7693407Z</Message></Error>


[None]

In [126]:
a = BlobSamples()
await a.download_multiple_blobs(blob_service_client=blob_service_client, container_name=CONTAINER_NAME, blob_names=image_blob_names)

1
An error occurred while downloading the blob raspberry_pi_uploads/1736779848.6032581(1).jpg: object StorageStreamDownloader can't be used in 'await' expression


[None]

In [20]:
image_blob_names

['raspberry_pi_uploads/1736779848.6032581(1).jpg']